This Capstone project was submitted in partial fulfillment of the [Google Advanced Data Analytics Certificate](https://www.coursera.org/specializations/google-advanced-data-analytics). 

# Project Overview

## About the company
Salifort Motors is a fictional French-based alternative energy vehicle manufacturer. Its global workforce of over 100,000 employees research, design, construct, validate, and distribute electric, solar, algae, and hydrogen-based vehicles. Salifort’s end-to-end vertical integration model has made it a global leader at the intersection of alternative energy and automobiles

## Business Case
Analyze data from a recent employee survey to come up with ideas for how to increase employee retention. Design a model that predicts whether an employee will leave the company based on their  department, number of projects, average monthly hours, and any other data points. 


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

import pickle

In [ ]:
df0 = pd.read_csv("/kaggle/input/salifort-motors-datasets/Files/HR_capstone_dataset.csv")
df0.head()

# Data Exploration (Initial EDA and data cleaning)

In [ ]:
df0.info()

In [ ]:
df0.describe()

In [ ]:
df0.columns

In [ ]:
df0= df0.rename(columns={'average_montly_hours': 'average_monthly_hours', 'time_spend_company': 'tenure', 
                         'Work_accident': 'work_accident', 'Department': 'department'})

df0.head()

In [ ]:
df0.isnull().sum()

In [ ]:
df0.duplicated().sum()

In [ ]:
df0[df0.duplicated()].head(10)

In [ ]:
df1= df0.drop_duplicates(keep='first')
df1.head()

In [ ]:
plt.title('Boxplot to detect outliers for tenure', fontsize=12)
sns.boxplot(x=df1['tenure'])
plt.show()

In [ ]:
# Determine the number of rows containing outliers

percentile25 = df1['tenure'].quantile(0.25)

percentile75 = df1['tenure'].quantile(0.75)

iqr = percentile75 - percentile25

upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr
print("Lower limit:", lower_limit)
print("Upper limit:", upper_limit)

outliers = df1[(df1['tenure'] > upper_limit) | (df1['tenure'] < lower_limit)]

print("Number of rows in the data containing outliers in `tenure`:", len(outliers))


In this initial EDA, I got to know the data and did some basic data cleaning. There were no missing values in the data.

However, some of the rows were duplicated. Are these legitimate entries, one might wonder? I could perform a likelihood analysis by essentially applying Bayes' theorem and multiplying the probabilities of finding each value in each column, but this does not seem necessary. With several continuous values across the first 10 columns, it doesn't seem like a legitimate entry, so I proceeded by dropping them. The duplicated rows account for approximately 20% of the data.

Upon checking the distribution of data for the variable 'tenure', there are 824 rows of data outliers. I wouldn't want to do anything to them since they are not much, and I intend to use Random Forest classification models, which handle a certain degree of outliers. 

# Data Exploration (Analyzing Relationships between Variables)

In [ ]:
print(df1['left'].value_counts())
print(df1['left'].value_counts(normalize=True)*100)

In [ ]:
import warnings
warnings.filterwarnings("ignore")



In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (22,8))

# boxplot showing `average_monthly_hours` distributions for `number_project`, comparing employees who stayed versus those who left
sns.boxplot(data=df1, x='average_monthly_hours', y='number_project', hue='left', orient="h", ax=ax[0])
ax[0].invert_yaxis()
ax[0].set_title('Monthly hours by number of projects', fontsize='14')

#  histogram showing distribution of `number_project`, comparing employees who stayed versus those who left
tenure_stay = df1[df1['left']==0]['number_project']
tenure_left = df1[df1['left']==1]['number_project']
sns.histplot(data=df1, x='number_project', hue='left', multiple='dodge', shrink=2, ax=ax[1])
ax[1].set_title('Number of projects histogram', fontsize='14')

plt.show()

From the plot above,

- Everyone with seven projects left the company. They worked the most hours with those who had six projects and also left, with some working nearly 300 hours a month.

- People who left the company can be divided into two groups. The first group worked less, while the second group worked more on the same number of projects. The first group might include people who got fired or possibly those who were assigned smaller tasks because they are already on their way out. It might be a valid reason to say that the second group quit, owing to the stress of having too many projects and working too many hours.

- People who had 3 or 4 projects not only have optimal working hours, but also have a lower left/stayed ratio.

- Almost everyone in the company is overworked if you consider the average hours they are supposed to work in a month. Assuming there are _52 weeks in a year and 2 weeks are for vacations, then we expect 50 weeks * 40 hours per week / 12 months = 166.67 hours per month_.

It would be wise to get the number of people who had 7 projects, since all of them left. 


In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (22,8))

# boxplot showing distributions of `satisfaction_level` by tenure, comparing employees who stayed versus those who left
sns.boxplot(data=df1, x='satisfaction_level', y='tenure', hue='left', orient="h", ax=ax[0])
ax[0].invert_yaxis()
ax[0].set_title('Satisfaction by tenure', fontsize='14')

# histogram showing distribution of `tenure`, comparing employees who stayed versus those who left
tenure_stay = df1[df1['left']==0]['tenure']
tenure_left = df1[df1['left']==1]['tenure']
sns.histplot(data=df1, x='tenure', hue='left', multiple='dodge', shrink=5, ax=ax[1])
ax[1].set_title('Tenure histogram', fontsize='14')

plt.show();

From this plot, we can see that 

- Again, two groups of people left: dissatisfied employees with shorter tenures and highly satisfied employees with medium-length tenures.
  
- There is an unusual satisfaction level data distribution for those four-year employers who left. It would be wise to investigate by checking with the company if possible.

- The satisfaction level of high-tenured and newer employees who stayed is relatively high. 

In [ ]:
df1.groupby(['left'])['satisfaction_level'].agg([np.mean,np.median])

As expected, people who stayed have higher average and median satisfaction levels compared to those who left. It is also important to note that the average satisfaction level of those who stayed is slightly lower than the median, suggesting that the satisfaction levels of those who stayed are skewed to the left. 

In [ ]:
#plot to examine relationship between `average_monthly_hours` and `promotion_last_5years`
plt.figure(figsize=(16, 3))
sns.scatterplot(data=df1, x='average_monthly_hours', y='promotion_last_5years', hue='left', alpha=0.4)
plt.axvline(x=166.67, color='#ff6361', ls='--')
plt.legend(labels=['166.67 hrs./mo.', 'left', 'stayed'])
plt.title('Monthly hours by promotion last 5 years', fontsize='14');

From the above plot,

- Only a few employees were promoted, and very few employers who worked long hours were promoted in the last 5 years.

- Employees who left worked the longest hours. 

In [ ]:
department_pct = (
    df1.groupby(['department', 'left']).size()
    .groupby(level=0).apply(lambda x: x / x.sum())
    .unstack(fill_value=0)
)

department_pct.plot(kind='bar', stacked=True, figsize=(8, 5))
plt.title('Percentage of Stayed/Left by Department', fontsize='10')
plt.ylabel('Proportion of Employees')
plt.xlabel('Department')
plt.xticks(rotation=75)
plt.legend(title='Left Company', labels=['Stayed', 'Left'])
plt.tight_layout()
plt.show()


There is a uniform proportion of those who left/stayed across all departments. 

In [ ]:
palette = {0: "#1f77b4", 1: "#ff7f0e"}  
g = sns.FacetGrid(df1, col="left", hue="left", palette=palette, height=5)
g.map_dataframe(sns.scatterplot, x='average_monthly_hours', y='last_evaluation', alpha=0.5)
g.set_axis_labels("Average Monthly Hours", "Last Evaluation Score")
g.add_legend(title="Left the Company")
g.fig.suptitle('Monthly Hours vs Evaluation: Stayed vs Left', y=1.05)


From the plot,

- The two groups of people who left show that one group was overworked but had high evaluation scores, while the other group worked slightly below the normal average hours but received poor evaluation scores.

- Hours worked and the evaluation score seem to correlate.

- Most of the employees worked more than 167 hours per month.

In [ ]:
plt.figure(figsize=(16, 9))
heatmap = sns.heatmap(df1.drop(['department', 'salary'], axis=1).corr(), vmin=-1, vmax=1, annot=True, cmap=sns.color_palette("vlag", as_cmap=True))
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':14}, pad=12);

The correlation heatmap indicates that the monthly hours, the number of projects worked, and the satisfaction levels all exhibit a positive correlation. The tendency of an employee to leave is negatively correlated with their satisfaction levels.

A major insight from this EDA is that management is clearly poor. Many employees left because of longer working hours, multiple projects, and generally low satisfaction scores. It is also unfavorable to work these many hours and receive no promotions and low evaluation scores. Finally, those who spent more than six years are less likely to leave. 

# Model Building

There are many classification models available, but I prefer to use the Random Forest classification model because of its robustness and performance, as I mentioned earlier. Since there is a class imbalance in the dependent variable 'left', my priority is to tune the model to identify as many people who leave as possible. To focus on minimizing false negatives, I will use recall as the main evaluation metric when tuning and assessing the model.

To begin, we encode the categorical variables. 

In [ ]:
df_model = df1.copy()

df_model['salary'] = (
    df_model['salary'].astype('category')
    .cat.set_categories(['low', 'medium', 'high'])
    .cat.codes
)

df_model = pd.get_dummies(df_model, drop_first=True)

df_model.head()

Next, we split the data into three parts: training, validation, and test sets.

In [ ]:
X = df_model.drop('left', axis=1)
y = df_model['left']
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, test_size=0.25, stratify=y_tr, random_state=42)

In [ ]:
for x in [X_train, X_val, X_test]:
    print(len(x))

We continue by instantiating the model and running a grid search for hyperparameter tuning.  

In [ ]:
# Instantiate the random forest classifier
rf = RandomForestClassifier(random_state=42, class_weight='balanced')


# dictionary of hyperparameters to tune
cv_params = {'max_depth': [None],
             'max_features': [1.0],
             'min_samples_leaf': [2],
             'min_samples_split': [2],
             'n_estimators': [100, 200, 300, 400],
             }

# list of scoring metrics to capture
scoring = ['accuracy', 'precision', 'recall', 'f1']

# Instantiate the GridSearchCV object
rf_cv = GridSearchCV(rf, cv_params, scoring=scoring, cv=4, refit='recall')

Now we train the model.

In [ ]:
%%time
rf_cv.fit(X_train, y_train)

In [ ]:
rf_cv.best_params_

In [ ]:
rf_cv.best_score_

In [ ]:
def make_results(model_name:str, model_object, metric:str):
    '''
    Arguments:
        model_name (string): what you want the model to be called in the output table
        model_object: a fit GridSearchCV object
        metric (string): precision, recall, f1, or accuracy

    Returns a pandas df with the F1, recall, precision, and accuracy scores
    for the model with the best mean 'metric' score across all validation folds.
    '''

    # dictionary that maps input metric to actual metric name in GridSearchCV
    metric_dict = {'precision': 'mean_test_precision',
                   'recall': 'mean_test_recall',
                   'f1': 'mean_test_f1',
                   'accuracy': 'mean_test_accuracy',
                   }

    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)

    # Isolate the row of the df with the max(metric) score
    best_estimator_results = cv_results.iloc[cv_results[metric_dict[metric]].idxmax(), :]

    # Extract accuracy, precision, recall, and f1 score from that row
    f1 = best_estimator_results.mean_test_f1
    recall = best_estimator_results.mean_test_recall
    precision = best_estimator_results.mean_test_precision
    accuracy = best_estimator_results.mean_test_accuracy

    # Create table of results
    table = pd.DataFrame({'model': [model_name],
                          'precision': [precision],
                          'recall': [recall],
                          'F1': [f1],
                          'accuracy': [accuracy]
                          }
                         )

    return table

In [ ]:
results = make_results('RF cv', rf_cv, 'recall')
results

After training and cross-validation, we have a random forest model with optimum parameters. The mean recall, precision, f1, and accuracy scores across the validation folds are relatively high. 

The next step is to use the best random forest model to predict on the validation data. 

## Model Selection

In [ ]:
rf_val_preds = rf_cv.best_estimator_.predict(X_val)

In [ ]:
def get_test_scores(model_name:str, preds, y_test_data):
    '''
    Generate a table of test scores.

    In:
        model_name (string): Your choice: how the model will be named in the output table
        preds: numpy array of test predictions
        y_test_data: numpy array of y_test data

    Out:
        table: a pandas df of precision, recall, f1, and accuracy scores for your model
    '''
    accuracy = accuracy_score(y_test_data, preds)
    precision = precision_score(y_test_data, preds)
    recall = recall_score(y_test_data, preds)
    f1 = f1_score(y_test_data, preds)

    table = pd.DataFrame({'model': [model_name],
                          'precision': [precision],
                          'recall': [recall],
                          'F1': [f1],
                          'accuracy': [accuracy]
                          })

    return table

In [ ]:
rf_val_scores = get_test_scores('RF val', rf_val_preds, y_val)

# Append to the results table
results = pd.concat([results, rf_val_scores], axis=0)
results

Beautiful! The model performed well in predicting the validation data. Now, let's see how it will do on the test data. 

In [ ]:
rf_test_preds = rf_cv.best_estimator_.predict(X_test)

rf_test_scores = get_test_scores('RF test', rf_test_preds, y_test)

results = pd.concat([results, rf_test_scores], axis=0)
results

In [ ]:
cm = confusion_matrix(y_test, rf_test_preds, labels=rf_cv.classes_)

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=['active', 'left'])
disp.plot();

The model predicts more false negatives than false positives, meaning that some employees who are genuinely at risk of leaving or being fired are not identified by the model. However, the number of missed cases remains relatively small, showing that the model is still effective at capturing most at-risk employees.

It would be helpful to analyze the variables in the data that contributed to the predictions outcomes. 

In [ ]:
# Get feature importances
feat_impt = rf_cv.best_estimator_.feature_importances_

# Get indices of top 10 features
ind = np.argpartition(rf_cv.best_estimator_.feature_importances_, -10)[-10:]

# Get column labels of top 10 features 
feat = X.columns[ind]

# Filter `feat_impt` to consist of top 10 feature importances
feat_impt = feat_impt[ind]

y_df = pd.DataFrame({"Feature":feat,"Importance":feat_impt})
y_sort_df = y_df.sort_values("Importance")
fig = plt.figure()
ax1 = fig.add_subplot(111)

y_sort_df.plot(kind='barh',ax=ax1,x="Feature",y="Importance")

ax1.set_title("Random Forest: Feature Importances for Employee Leaving", fontsize=12)
ax1.set_ylabel("Feature")
ax1.set_xlabel("Importance")

plt.show()

From the plot above, it is clear that 'satisfaction_level', 'tenure', 'last_evaluation', 'average_monthly_hours', and 'number_project' had the greatest influence on predicting the outcome, respectively. 

# Results and Evaluation

## Summary of Model Results

The Random Forest model achieved a precision of 97.6%, a recall of 92.7%, an F1 score of 95.1%, and an accuracy of 98.4% on the test set.

## Conclusions and Recommendations

As pointed out in the EDA and also in the models' feature importance plot, the employees are overworked. 

To support employee retention, the following recommendations are proposed:

- Limit the number of projects assigned to each employee to help manage workload and reduce burnout.

- Review the experiences of employees who have been with the company for four years or more. Consider promoting them where appropriate or investigating potential reasons for dissatisfaction within this group.

- Create a fair approach to longer working hours. Either recognize and reward extended efforts or avoid making them a requirement.

- Ensure all employees clearly understand overtime policies and expectations around working hours and time off. Communicate these policies openly and make sure they are consistently applied.

- Encourage open conversations about the company culture. Hold discussions at both the company and team levels to understand concerns and identify areas for improvement.

- Avoid tying high evaluation scores only to employees who work excessively long hours. Use a performance recognition system that rewards effort, contribution, and quality of work, not just time spent.